## Example for model inference

In [32]:
import torch
import torch
from hydra import compose, initialize, initialize_config_dir
from omegaconf import OmegaConf
import hydra
import numpy as np


def load_model(config_path, checkpoint_path):
    """
    Utility function to load a model from a checkpoint.
    """

    with initialize_config_dir(version_base=None, config_dir=config_path, job_name="test_app"):
        cfg = compose(config_name="config.yaml", return_hydra_config=True, 
                    overrides=["data.multiprocessing_context=null", "data.batch_size=16","data.sampler_cls._target_=cellm.data.data_samplers.BaseSampler","+data.output_map.return_index=True"])
        print(OmegaConf.to_yaml(cfg))

    checkpoint = torch.load(checkpoint_path)
    metrics = hydra.utils.instantiate(cfg.get("metrics"))
    model = hydra.utils.instantiate(cfg.model, metrics = metrics)
    model.load_state_dict(checkpoint["state_dict"])
    model.eval()

    cfg.paths.output_dir = ""

    return model

class ForwardModel(torch.nn.Module):
    """
    A wrapper class for the model to handle the forward pass.
    If last layer = True, it returns the last layer of the embedding

    Output :
    - patient embedding
    - cell cross embedding
    - patient prediction
    """
    def __init__(self, base_model, last_layer = True):
        super().__init__()
        self.base_model = base_model
        self.last_layer = last_layer
    def forward(self, x, padding_mask):
        #assert x.shape[0] == 1
        cell_embds = self.base_model.gene2cell_encoder(x)
        cell_cross_embds = self.base_model.cell2cell_encoder(cell_embds, padding_mask = padding_mask)
        patient_embds = self.base_model.cell2patient_aggregation.aggregate(data = cell_cross_embds, mask = padding_mask)
        patient_embds_2 = self.base_model.patient_encoder(patient_embds)
        patient_preds = self.base_model.patient_predictor(patient_embds_2)
        if self.last_layer:
            return patient_embds_2, cell_cross_embds, patient_preds
        else:
            return patient_embds, cell_cross_embds, patient_preds


def lognormalize(x, padded_mask, target_sum = 1e4):
    """
    Normalize the input tensor using log normalization.
    """
    X = x
    pad_mask = padded_mask

    counts_per_cell = X.sum(axis=-1) +1e-8
    counts_per_cell = counts_per_cell / target_sum

    counts_per_cell[~pad_mask] = 1

    X_padded_norm = X / counts_per_cell[..., None]

    X_padded_out = X_padded_norm.log1p()

    return X_padded_out, padded_mask

In [30]:
run_name = "2025-02-09_22-25-15"
model_name = "epoch_002" 
config_path = f"/homefs/home/debroue1/projects/cellm/logs/train/runs/{run_name}/.hydra/"
checkpoint_path = f"/homefs/home/debroue1/projects/cellm/logs/train/runs/{run_name}/checkpoints/{model_name}.ckpt"
model = load_model(config_path, checkpoint_path)
model_fwd = ForwardModel(model, last_layer = True)

hydra:
  run:
    dir: outputs/${now:%Y-%m-%d}/${now:%H-%M-%S}
  sweep:
    dir: multirun/${now:%Y-%m-%d}/${now:%H-%M-%S}
    subdir: ${hydra.job.num}
  launcher:
    _target_: hydra._internal.core_plugins.basic_launcher.BasicLauncher
  sweeper:
    _target_: hydra._internal.core_plugins.basic_sweeper.BasicSweeper
    max_batch_size: null
    params: null
  help:
    app_name: ${hydra.job.name}
    header: '${hydra.help.app_name} is powered by Hydra.

      '
    footer: 'Powered by Hydra (https://hydra.cc)

      Use --hydra-help to view Hydra specific help

      '
    template: '${hydra.help.header}

      == Configuration groups ==

      Compose your configuration from those groups (group=option)


      $APP_CONFIG_GROUPS


      == Config ==

      Override anything in the config (foo.bar=value)


      $CONFIG


      ${hydra.help.footer}

      '
  hydra_help:
    template: 'Hydra (${hydra.runtime.version})

      See https://hydra.cc for more info.


      == Flags ==

      

/tmp/ipykernel_3028542/3133030091.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)
/homefs/home/debroue1/miniforge3/envs/cellm/l

In [ ]:
from cellm.data.data_structures import SampleBatch


# Example for orginal count data
# Tensor size should be Samples x 1 x Cells x Genes
x = 50 + torch.randn(16,1,1000,28231)
# Padding mask is True if cell is observed and False if cell is masked
padding_mask = torch.ones(16,1,1000).bool()

x, padding_mask = lognormalize(x, padding_mask)
embeds = model_fwd(x, padding_mask)

In [34]:
embeds

(tensor([[[-0.8152, -0.5030,  2.2511,  ..., -0.4717, -1.4045,  0.2516]],
 
         [[-0.8150, -0.5032,  2.2534,  ..., -0.4715, -1.4041,  0.2485]],
 
         [[-0.8152, -0.5028,  2.2525,  ..., -0.4714, -1.4039,  0.2510]],
 
         ...,
 
         [[-0.8151, -0.5028,  2.2508,  ..., -0.4715, -1.4038,  0.2540]],
 
         [[-0.8155, -0.5030,  2.2523,  ..., -0.4718, -1.4046,  0.2520]],
 
         [[-0.8153, -0.5032,  2.2524,  ..., -0.4716, -1.4041,  0.2526]]],
        grad_fn=<PreluKernelBackward0>),
 tensor([[[[-3.4786,  1.5765,  1.3317,  ...,  1.4100,  2.8394, -2.6609],
           [-3.5124,  1.5820,  1.3329,  ...,  1.4465,  2.8304, -2.6920],
           [-3.5021,  1.6246,  1.3763,  ...,  1.4691,  2.8533, -2.6118],
           ...,
           [-3.4855,  1.5913,  1.3168,  ...,  1.4305,  2.8709, -2.7046],
           [-3.4916,  1.5983,  1.2849,  ...,  1.4268,  2.8336, -2.7094],
           [-3.5350,  1.5697,  1.2935,  ...,  1.3962,  2.8940, -2.6702]]],
 
 
         [[[-3.5054,  1.5711,  1.3